# NLP analysis objectives
- switch to outscraper to extract full information of reviews ✓
- check if the place might have water army?
- extract the most related
- use sentiment analysis to analysis the score rather than using google score
- NLTK

# To Do:
- download a set of data with only reviews and rating or Aurther ✓
- Word2vec to show its true reviews
- sentiment analysis to measure its true value

# Tech issue
- maybe code a function to download necessary nlp trained model

In [1]:
import pandas as pd
from nltk.corpus import stopwords
import nltk
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.tokenize import word_tokenize
import numpy as np
import re,string
from gensim.models import Word2Vec
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /home/aipen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/aipen/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
df=pd.read_csv('data_references/restuarant_data1.csv')
df=df[['review','rating']]
df

,review,rating
0,"Craving for some good food, I been very surpri...",4
1,I haven't been for a time as the last time I w...,5
2,Had a great meal celebrating our daughters gra...,5
3,First and also my last time at this restaurant...,1
4,I really loved my visit here in this lovely re...,5
5,Came here today for lunch for my birthday.I ha...,2
6,"Great food, lovely place to eat, and really lo...",5
7,Great food (although could have been a bit war...,3
8,"Menu on the outside said, before 18:00 two cou...",1
9,"The food was excellent, the staff was very nic...",5


In [3]:
def clean_data(word):
    model= stopwords.words('english')
    trim_s=WordNetLemmatizer()
    word = ''.join([i for i in str(word) if not i.isdigit()])
    word = word_tokenize(word.lower())
    word = [i for i in set(word) if i.isalpha() and i not in string.punctuation and i not in model]
    return " ".join(word)

In [4]:
df['true_review']=np.vectorize(clean_data)(df.review)

In [5]:
df.head()

,review,rating,true_review
0,"Craving for some good food, I been very surpri...",4,texture bistro weird door meal spot book find ...
1,I haven't been for a time as the last time I w...,5,last minor medallions quality trip gives favou...
2,Had a great meal celebrating our daughters gra...,5,recommended graduation meal bourguignon mushro...
3,First and also my last time at this restaurant...,1,course nothing ate also tables photo space lit...
4,I really loved my visit here in this lovely re...,5,works hungry really worth city excellent loved...


In [6]:
X=df['true_review']
y=df['rating']

In [7]:
word2vec = Word2Vec(sentences=X, vector_size=60, min_count=1, window=10)

In [8]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Function to convert a sentence (list of words) into a matrix representing the words in the embedding space
def embed_sentence(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec.wv:
            embedded_sentence.append(word2vec.wv[word])
        
    return np.array(embedded_sentence)

# Function that converts a list of sentences into a list of matrices
def embedding(word2vec, sentences):
    embed = []
    
    for sentence in sentences:
        embedded_sentence = embed_sentence(word2vec, sentence)
        embed.append(embedded_sentence)
        
    return embed

# Embed the training and test sentences
X = embedding(word2vec, X)
X = pad_sequences(X,dtype='float32', padding='post', maxlen=200)

2022-06-08 22:24:23.636537: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-08 22:24:23.636590: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [9]:
len(X[1])

200

In [10]:
from tensorflow.keras import Sequential
from tensorflow.keras import layers

def init_model():
    model = Sequential()
    model.add(layers.Masking())
    model.add(layers.LSTM(20, activation='tanh'))
    model.add(layers.Dense(15, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])
    
    return model
model=init_model()

2022-06-08 22:24:25.589293: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:925] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-06-08 22:24:25.589420: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-08 22:24:25.589465: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-06-08 22:24:25.589495: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-06-08 22:24:25.589524: W tensorflow/stream_executor/platform/default/dso_loader.cc:6

In [11]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=5, restore_best_weights=True)

model.fit(X, y, 
          batch_size = 32,
          epochs=5,
          callbacks=[es]
         )

Epoch 1/5
1/1 [==============================] - 2s 2s/step - loss: 1.0165 - accuracy: 0.0000e+00
Epoch 2/5
1/1 [==============================] - 0s 68ms/step - loss: 0.1750 - accuracy: 0.2000
Epoch 3/5
1/1 [==============================] - 0s 66ms/step - loss: -0.1862 - accuracy: 0.2000
Epoch 4/5
1/1 [==============================] - 0s 63ms/step - loss: -0.5725 - accuracy: 0.2000
Epoch 5/5
1/1 [==============================] - 0s 66ms/step - loss: -0.9024 - accuracy: 0.2000


In [13]:

!git add NLP.ipynb
!git commit -m 'built a base model, need to adjust accordingly'
!git push origin master


[master f487cf1] built a base model, need to adjust accordingly
 1 file changed, 364 insertions(+), 52 deletions(-)
Enumerating objects: 5, done.
Counting objects: 100% (5/5), done.
Delta compression using up to 20 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 5.25 KiB | 5.25 MiB/s, done.
Total 3 (delta 1), reused 0 (delta 0)
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To https://github.com/Aipeni/map_custom.git
   806bc6a..f487cf1  master -> master
